In [1]:
!pip install konlpy

In [2]:
import nltk
from konlpy.tag import Okt
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
okt = Okt()

In [3]:
economy_df = pd.read_csv("./economy_df.csv",index_col=0)
culture_df = pd.read_csv("./culture_df.csv",index_col=0)
politics_df = pd.read_csv("./politics_df.csv",index_col=0)

In [5]:
news_df = pd.concat([economy_df,culture_df,politics_df])
news_df = news_df.reset_index(drop=True)
news_df

,name,url,sections,Sectors,content
0,"수협은행, 디지털 오피스 구현",https://n.news.naver.com/mnews/article/014/000...,금융,경제,강신숙 수협은행장이 21일 온라인 커뮤니케이션 시스템으로 다자간 영상 컨퍼런스 콜을...
1,"KB증권, '프리미어 서밋' 11월까지 개최",https://n.news.naver.com/mnews/article/003/001...,금융,경제,KB증권 *재판매 및 DB 금지[서울=뉴시스] 강수윤 기자 = KB증권은 22일부...
2,"한은 총재 ""물가 불확실성 한두달 새 커져…美 빅스텝 예상도""",https://n.news.naver.com/mnews/article/421/000...,금융,경제,"국회 기재위 현안보고…""통화정책 정교히 운영해야""\n\n\n\n이창용 한국은행 총재..."
3,2월 부산 소비자심리지수 하락…공공요금 인상 영향,https://n.news.naver.com/mnews/article/001/001...,금융,경제,한국은행 부산본부[촬영 조정호] (부산=연합뉴스) 조정호 기자 = 물가상승 우...
4,"DGB대구은행, 경북 소상공인에 대출지원 자금 60억 출연",https://n.news.naver.com/mnews/article/009/000...,금융,경제,신용보증재단 통해 900억원 대출소기업·소상공인 연 2% 이자지원\n\n\n\n 2...
...,...,...,...,...,...
7179,소양강댐 발전시설 준공 50년 만의 첫 외부개방 결정,https://n.news.naver.com/mnews/article/087/000...,정치일반,정치,-한기호 국방위원장 제안에 한국수자원공사 수용-10월 소양강댐 50주년 행사와 연계...
7180,"김기현, 대세론 굳히나...'중도 보수 끌어안기'",https://n.news.naver.com/mnews/article/629/000...,정치일반,정치,안철수 향해 '보수 정체성' 공세...전통적 보수층 공략 이어가'친윤 없는 안정감'...
7181,"與 '진흙탕' 2차 토론...""울산의 이재명"" ""정치생명 걸어라""",https://n.news.naver.com/mnews/article/629/000...,정치일반,정치,"김기현 '울산 KTX 의혹' 난타전천하람·황교안, 투기 의혹 '맹폭'컨벤션효과? 역..."
7182,李 정의당에 '노란봉투법' 손길?…김건희 특검 이견 여전,https://n.news.naver.com/mnews/article/629/000...,정치일반,정치,"""노란봉투법 조속한 처리 최선""...농성장 방문까지 정의당 '50억 클럽 특검' 발..."


In [6]:
news_df = news_df.drop_duplicates("url")
news_df = news_df.dropna()
news_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5133 entries, 0 to 6887
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      5133 non-null   object
 1   url       5133 non-null   object
 2   sections  5133 non-null   object
 3   Sectors   5133 non-null   object
 4   content   5133 non-null   object
dtypes: object(5)
memory usage: 240.6+ KB


In [6]:
pos_map = {}
for i in tqdm(news_df["content"].index):
    text = news_df.loc[i, "content"]
    poss = okt.pos(text)
    pos_map[i] = poss

pos_map

Output hidden; open in https://colab.research.google.com to view.

In [7]:
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

### noun, adjective, verb, adverb빼오기


In [8]:
#딕셔너리 안에 리스트 넣기
word_class = {}

#행별 기사 가져오기
for i in tqdm(news_df["content"].index):
     text = pos_map[i]
     
#word_class[i]리스트에 해당하는 품사 넣기 
     word_class[i] = []
     for word,tagset in text:
        if tagset in ['Adjective','Adverb','Noun','Verb']:
            word_class[i].append(word)

100%|██████████| 5133/5133 [00:01<00:00, 3970.66it/s]


### 중복값 제거하기

In [9]:
#리스트 만들고
#value값만 for문으로 가져와서
#하나씩 extend로 리스트에 추가
#list_set중복 제거

word_list = []
for value in word_class.values():
    word_list.extend(value)
    word_list = list(set(word_list)) #set중복제거, list로 만들기

print(word_list)

['다단계', '김두식', '주석', '게이고', '연등동', '임성훈', '삼양식품', '지나치다는', '따라가야', '대사성', '수소', '치매', '살펴보려', '스타디움', '신성균', '참사', '뮌스터', '새로도', '집비둘기', '싫은데', '없기에', '개념', '환경공학', '건너는', '나희', '방염', '쉽', '대일', '비욘드', '두드러졌습니다', '허다해', '숙련', '추운데요', '피렌체', '등식', '바이든', '아니다란', '안이하게', '약해질', '기씨', '영연방', '단정한', '미식', '추천사', '모씨', '호성', '거제', '레지던스', '와서', '스러', '송어', '엑셀', '겉멋', '맹신', '해양력', '쉬', '식탐', '행정관청', '향연', '김창훈', '얼세대', '놓쳐한', '있지요', '길게', '김자영', '될지가', '꿈꾸지만', '넘어섰다는', '스며든', '밴으로', '짚는', '주목', '스쿼트', '치열해질', '높아진다는', '자연히', '전해진', '계면', '복무', '내려졌는데요', '악용', '가볍다는', '떼내고', '센텀', '약하겠다는', '습기', '무빙', '악화시키', '라나', '길어졌다', '들지만', '박형근', '장재', '반가웠다', '두진', '한겨레신문사', '돌리려고', '초속', '양현아', '돌발', '손발', '바라봐야', '저수지', '지르는', '실밥', '면밀하게', '김상훈', '금산사', '평담청', '꾸준해', '홉', '상징물', '엇갈리는', '드레스', '단백', '생년', '아끼지', '물질', '레드카펫', '갉아먹었던', '감산', '소호', '이나진', '게임화', '칠해', '연고', '져도', '싶어지는', '통관', '뿌리', '되면', '안타깝다', '물려서', '커런트', '굽어', '내려갔고', '엄청', '해반드르', '본사', '성인', '기르는데', '일산화탄소', '신호', '서까지', '하우

In [10]:
print("전체 데이터 row 수 : ",len(word_class))
print("중복 제거된 단어 개수 : ",len(word_list))

전체 데이터 row 수 :  5133
중복 제거된 단어 개수 :  43890


In [11]:
#처음 index로 for문 돌려서 시리즈로 만들기
#중복이 된거 가져와서 시리즈로 만들기
#value_count해주기
#loc사용해서 value_count한거에 value값만 넣기
#NAN을 0으로 만들기
#품사 개수 value값을 시리즈로 만들고

In [12]:
word_list_df = pd.DataFrame(columns=word_list)
word_list_df

,다단계,김두식,주석,게이고,연등동,임성훈,삼양식품,지나치다는,따라가야,대사성,...,인적,박차정,낮췄고,가식,분자,오래,피츠너,신설,트러스트,균수


In [13]:
#series test
pd.Series(word_class[3]).value_counts().values

array([9, 6, 6, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1])

In [14]:
# for i in tqdm(news_df.sample(100)['content'].index):
#     word_class_series = pd.Series(word_class[i]).value_counts()
#     word_list_df.loc[i,word_class_series.index] = word_class_series.values

In [15]:
datas = []
for i in tqdm(news_df['content'].index):
    word_class_series = pd.Series(word_class[i]).value_counts()
    data = word_class_series.to_dict()
    datas.append(data)
    # word_list_df.loc[i,word_class_series.index] = word_class_series.values

word_list_df = pd.DataFrame(datas, index=news_df.index)

 55%|█████▌    | 2828/5133 [00:02<00:01, 1253.71it/s]<ipython-input-15-181261feff87>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  word_class_series = pd.Series(word_class[i]).value_counts()
100%|██████████| 5133/5133 [00:05<00:00, 1007.19it/s]


In [16]:
word_list_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5133 entries, 0 to 6887
Columns: 43890 entries, 은행 to 신사우동
dtypes: float64(43890)
memory usage: 1.7 GB


### fillna로 결측값 처리

In [17]:
# word_list_df = word_list_df.copy()
word_list_nan = word_list_df.fillna(0)
word_list_nan

,은행,수협,온라인,시스템,커뮤니케이션,금융,했다,공유,통해,주요,...,야하지,심다지기에,운전기사,갑당,희기자,사우,신북읍,사북면,북산면,신사우동
0,6.0,5.0,5.0,5.0,4.0,4.0,4.0,3.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6466,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6467,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6468,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6469,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
word_list_nan.to_csv('bow.csv')

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# !cp ./drive/MyDrive/bow.csv ./bow.csv
word_list_nan = pd.read_csv("bow.csv", index_col=0)

### Section추가

In [7]:
#섹션 컬럼 붙이기
word_list_nan["Sectors"] = news_df["Sectors"]

In [33]:
#섹션 명 0,1,2,3으로 변환 시키기 replace
# word_list_nan.replace([["경제",0],["정치",1],["생활문화 일반",2]], inplace=True)

unique_sectors = word_list_nan["Sectors"].unique()

sector_map = {}
sector_unmap = {}

for i, sector in enumerate(unique_sectors):
    sector_unmap[i] = sector
    sector_map[sector] = i

print(sector_map)
print(sector_unmap)

{0: 0, 1: 1, 2: 2}
{0: 0, 1: 1, 2: 2}


In [15]:
word_list_nan["Sectors"] = word_list_nan["Sectors"].replace(sector_map)

In [17]:
# word_list_nan["Sectors"].replace(sector_unmap)

In [16]:
news_df["Sectors"]

0       경제
1       경제
2       경제
3       경제
4       경제
        ..
6466    정치
6467    정치
6468    정치
6469    정치
6887    정치
Name: Sectors, Length: 5133, dtype: object

In [18]:
word_list_nan

,은행,수협,온라인,시스템,커뮤니케이션,금융,했다,공유,통해,주요,...,심다지기에,운전기사,갑당,희기자,사우,신북읍,사북면,북산면,신사우동,Sectors
0,6.0,5.0,5.0,5.0,4.0,4.0,4.0,3.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,11.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6466,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
6467,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
6468,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
6469,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


### Random Forest

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = word_list_nan.drop("Sectors", axis=1) #sectors빼고 출력
y = word_list_nan['Sectors']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)

rf = RandomForestClassifier(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [20]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
print('accuracy_score : ',accuracy_score(y_test,y_pred))

accuracy_score :  0.861038961038961


In [32]:
#섹션별 비율
bow = word_list_nan
all_len = len(bow["Sectors"])
print('전체 길이:',all_len)
vc = bow["Sectors"].value_counts()
vc = vc / all_len
vc.index = pd.Series(vc.index).replace(sector_unmap)
vc

전체 길이: 5133
{0: 0, 1: 1, 2: 2}


0    0.734074
1    0.259692
2    0.006234
Name: Sectors, dtype: float64

In [24]:
# economy_ratio = / all_len * 100
# social_ratio = / all_len * 100
# cluture_ratio = / all_len * 100
# politics_ratio = / all_len * 100

# print('economy_ratio :', economy_ratio)
# print('politics_ratio :', politics_ratio)
# print('social_ratio :', social_ratio)
# print('culture_ratio :', culture_ratio)

SyntaxError: ignored